In [1]:
library(dplyr) #%>%

Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# Download data

In [2]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
path_visual = file.path(path_, 'OFH_performance')
dir.create(path_visual)
#df = read.csv(file.path(path_,'OFH_df_all_mvt.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'df_eggdata.csv'), header = TRUE, sep = ",")
#factors
df$PenID = factor(df$PenID)
df$DIB = as.integer(df$DIB)
df$Treatment_allpens = factor(df$Treatment_allpens) #unsacaled: 0-1
#setting reference group
df <- df %>% mutate(Treatment_allpens = relevel(Treatment_allpens, ref = "TRAN"))
contrasts(df$Treatment_allpens)
print(dim(df))
summary(df)
head(df,3)

Warning message in dir.create(path_visual):
"'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput\TrackingSystem\ALLDATA_\OFH_performance' existe déjà"


,OFH
TRAN,0
OFH,1


[1] 3360   10


     PenID              Treatment            Date           DIB        
 3      : 336   OFH          :1344   2020-09-24:  10   Min.   : -5.00  
 4      : 336   OFH-nonfocal : 336   2020-09-25:  10   1st Qu.: 78.75  
 5      : 336   TRAN         :1344   2020-09-26:  10   Median :162.50  
 6      : 336   TRAN-nonfocal: 336   2020-09-27:  10   Mean   :162.50  
 7      : 336                        2020-09-28:  10   3rd Qu.:246.25  
 8      : 336                        2020-09-29:  10   Max.   :330.00  
 (Other):1344                        (Other)   :3300                   
      WIB         Tier.bestand   X.eggOutsideBoxPerTier  X.eggPerTier   
 Min.   : 0.00   Min.   :197.0   Min.   :0.00000        Min.   :0.0000  
 1st Qu.:12.00   1st Qu.:205.0   1st Qu.:0.00000        1st Qu.:0.9415  
 Median :24.00   Median :210.0   Median :0.00000        Median :0.9657  
 Mean   :23.64   Mean   :211.3   Mean   :0.00205        Mean   :0.8796  
 3rd Qu.:36.00   3rd Qu.:216.0   3rd Qu.:0.00000        3rd

,PenID,Treatment,Date,DIB,WIB,Tier.bestand,X.eggOutsideBoxPerTier,X.eggPerTier,Treatment_allpens,Early_death
,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<dbl>
1,3,OFH,2020-09-24,-5,0,225,0,0,OFH,0
2,3,OFH,2020-09-25,-4,0,225,0,0,OFH,0
3,3,OFH,2020-09-26,-3,0,225,0,0,OFH,0


In [3]:
#df = df[!(df$PenID %in% c(6,7)), ]

In [4]:
#choosing first two month in the laying barn
df_egg = df[(df$DIB<=60)&(df$DIB>=1),]

# Sigmoid curve

In [10]:
li = list()
tiff(file.path(path_visual, 'egg_growth_crv.tiff'), width=400, height=400)
df_OFH = df_egg[(df_egg$Treatment_allpens=='OFH'),]
df_TRAN = df_egg[(df_egg$Treatment_allpens=='TRAN'),]
plot(df_OFH$DIB, df_OFH$X.eggPerTier, col=rgb(red = 0, green = 0, blue = 1, alpha = 0.1))
points(df_TRAN$DIB, df_TRAN$X.eggPerTier, col=rgb(red = 1, green = 0, blue = 0, alpha = 0.1))
i = 1
for (penid in unique(df$PenID)){
    print('----------------------------------------------')
    print(penid)
    df_penid = df_egg[(df_egg$PenID==penid),]
    df_penid = arrange(df_penid, DIB, by_group = FALSE)
    color = rgb(red = 0, green = 0, blue = 1, alpha = 0.6)
    if (unique(df_penid$Treatment_allpens)[[1]]=='OFH'){color = rgb(red = 1, green = 0, blue = 0, alpha = 0.6)}
    head(df_penid,3)
    fit_penid = nls(X.eggPerTier ~ SSlogis(DIB, Asym, xmid, scal), data = df_penid) #SSlogis() logistic growth curve
    #Asym, xmid, scal are param that will be estiamted, you can name them as you want
    #from documentation (https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/SSlogis) :
    #Asym: a numeric parameter representing the asymptote (i.e. max posisble value)
    #xmid: a numeric param. representing the x value at the inflection pt of the crv. The value of SSlogis will be Asym/2 at xmid
    #scal: a numeric scale parameter on the input axis.
    print(summary(fit_penid))
    #"The logistic growth curve has some upper bound on it (i.e. some point which the model can not pass). The model estimates this
    #to be 1858.0956. So, if I were to make predictions for very large x, you would see that the curve will get very close to 25.657 but will never touch it or pass it. T
    #plot(df_penid$DIB, df_penid$X.eggPerTier)
    lines(df_penid$DIB, predict(fit_penid), col=color, lwd=1.2)  
    df_res = as.data.frame(summary(fit_penid)$coefficients)
    df_res$PenID = penid
    li[[i]] = df_res
    i = i + 1
}

dev.off()
df_res = rbind(li[[1]],li[[2]],li[[3]],li[[4]],li[[5]],li[[6]],li[[7]],li[[8]],li[[9]],li[[10]])
write.csv(df_res, file.path(path_visual,'egg_growthcurve.csv')) 
df_res

[1] "----------------------------------------------"
[1] "3"

Formula: X.eggPerTier ~ SSlogis(DIB, Asym, xmid, scal)

Parameters:
      Estimate Std. Error t value Pr(>|t|)    
Asym  0.951955   0.007562  125.89   <2e-16 ***
xmid 24.049881   0.207618  115.84   <2e-16 ***
scal  4.135419   0.178141   23.21   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.03493 on 57 degrees of freedom

Number of iterations to convergence: 0 
Achieved convergence tolerance: 6.61e-07

[1] "----------------------------------------------"
[1] "4"

Formula: X.eggPerTier ~ SSlogis(DIB, Asym, xmid, scal)

Parameters:
      Estimate Std. Error t value Pr(>|t|)    
Asym  0.949651   0.006041  157.21   <2e-16 ***
xmid 24.307380   0.162542  149.54   <2e-16 ***
scal  3.926349   0.139660   28.11   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.02822 on 57 degrees of freedom

Number of iterations to conv

png 
  2

,Estimate,Std. Error,t value,Pr(>|t|),PenID
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Asym,0.9519553,0.007562100,125.88506,2.100468e-71,3
xmid,24.0498806,0.207618372,115.83696,2.364101e-69,3
scal,4.1354192,0.178140870,23.21432,9.841249e-31,3
Asym1,0.9496510,0.006040662,157.20976,6.873369e-77,4
xmid1,24.3073797,0.162541803,149.54540,1.178945e-75,4
scal1,3.9263495,0.139659711,28.11369,4.255029e-35,4
Asym2,0.9643251,0.007607548,126.75899,1.417927e-71,5
xmid2,24.5622739,0.195264436,125.78980,2.192751e-71,5
scal2,3.6359598,0.168130168,21.62586,3.729646e-29,5


# death

path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
path_visual = file.path(path_, 'OFH_performance')
dir.create(path_visual)
#df = read.csv(file.path(path_,'OFH_df_all_mvt.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'df_death.csv'), header = TRUE, sep = ",")
#factors
df$PenID = factor(df$PenID)
df$DIB = as.integer(df$DIB)
df$Treatment_allpens = factor(df$Treatment_allpens) #unsacaled: 0-1
#setting reference group
df <- df %>% mutate(Treatment_allpens = relevel(Treatment_allpens, ref = "TRAN"))
contrasts(df$Treatment_allpens)
df$DIB_rs = rescale(df$DIB)
#two_sd: variables are scaled by two times the deviation (SD or MAD depending on robust). 
#This method can be useful to obtain model coefficients of continuous parameters comparable to coefficients related to binary predictors (Gelman, 2008).
#scale var1 to have mean = 0 and standard deviation = 0.5
#rescale
print(dim(df))
summary(df)
head(df,3)

df_ = df[(df$'DIB'<=60)&(df$'DIB'>=1),]
dim(df_)
sum(df_$Early_death)
fit = glmmTMB(Early_death ~  Treatment_allpens+rescale(DIB) +(1|PenID), data=df_, ziformula = ~rescale(DIB),family=binomial)
simulationOutput = simulateResiduals(fittedModel=fit)
#cvg probleme without dib in ziformula
plot(simulationOutput)
summary(fit)
testZeroInflation(simulationOutput)

#library(survival)
#https://boostedml.com/2018/10/cox-regression-the-basic-idea.html
#https://cran.r-project.org/web/packages/survival/vignettes/survival.pdf
#https://rdrr.io/cran/survival/man/coxph.html
#https://www.rdocumentation.org/packages/survival/versions/2.11-4/topics/Surv
fit = coxph(Surv(Early_death) ~ Treatment_allpens + DIB, data=df_)
summary(fit)
coxph.detail(fit)